In [2]:
!pip install transformers bitsandbytes accelerate datasets peft trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 16.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.1/80.1 kB 5.9 MB/s eta 0:00:00


In [3]:
from datasets import Dataset, load_dataset

dataset = load_dataset(
    'json',
    data_files={
        'train':
        '../input/nlp-project/nlp_dataset/train.txt',
        'validation':
        '../input/nlp-project/nlp_dataset/dev.txt',
        'test':
        '../input/nlp-project/nlp_dataset/test.txt',
    }
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0be3e950eaeb0bc2/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model_name = 'fnlp/bart-base-chinese'

In [5]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/259k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.69k [00:00<?, ?B/s]

In [6]:
# Ensure the tokenizer recognizes a pad token
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [7]:
def create_training_examples(conversations):
    inputs, targets = [], []  # Initialize the lists for inputs and targets
    for conversation in conversations:
        for i in range(1, len(conversation), 2):  # Iterate over bot responses
            # Validate the alternating pattern: the message before a bot response should be from the user
            if i - 1 >= 0 and conversation[i - 1].startswith("user:"):
                # Prepare context with up to 8 most recent turns, ensuring it ends with a user's message
                start_idx = max(0, i - 15)  # Adjust to capture up to 8 turns
                context = conversation[start_idx:i]  # Exclude the bot response itself from the context

                # Format each turn in the context
                formatted_context = []
                for turn in context:
                    # Determine the prefix and remove "user:"/"bot:" from the text
                    if turn.startswith("user:"):
                        prefix = "\nInstruction:\n"  # Add newline before each "Instruction:" for proper formatting
                    else:
                        prefix = "\nResponse:\n"
                    turn_text = turn.split(': ', 1)[1] if ': ' in turn else turn
                    formatted_context.append(f"{prefix}{turn_text}")

                # Join the formatted context turns into a single input string
                input_text = "".join(formatted_context).lstrip('\n')  # Remove leading newline

                # Remove the "bot:" prefix from the target and format it
                target_text = conversation[i].split(': ', 1)[1] if ': ' in conversation[i] else conversation[i]

                inputs.append(input_text)
                targets.append(target_text)

    return {"input": inputs, "target": targets}

# Apply the function to each conversation in the dataset splits
processed_datasets = {split: Dataset.from_dict(create_training_examples(dataset[split]['conversation'])) for split in dataset.keys()}

In [133]:
print('###input:',processed_datasets['train']['input'][0])
print('###output:',processed_datasets['train']['target'][0])

###input: Instruction:
你好，今天是几号了？
###output: 你好，今天是2018年1月18日。


In [134]:
print('###input:',processed_datasets['train']['input'][1])
print('###output:',processed_datasets['train']['target'][1])

###input: Instruction:
你好，今天是几号了？
Response:
你好，今天是2018年1月18日。
Instruction:
谢谢你，我都给忙忘记了。
###output: 哈哈，我还知道今天是周杰伦的生日呢。


In [135]:
print('###input',processed_datasets['train']['input'][2])
print('###output:',processed_datasets['train']['target'][2])

###input Instruction:
你好，今天是几号了？
Response:
你好，今天是2018年1月18日。
Instruction:
谢谢你，我都给忙忘记了。
Response:
哈哈，我还知道今天是周杰伦的生日呢。
Instruction:
真的吗？我最喜欢听他的歌了，他是我的偶像呢。
###output: 真的呢，他的歌声确实是很独特，还获得过四届新加坡金曲奖最受欢迎男歌手奖呢。


In [136]:
print('###input',processed_datasets['train']['input'][3])
print('###output:',processed_datasets['train']['target'][3])

###input Instruction:
你好，今天是几号了？
Response:
你好，今天是2018年1月18日。
Instruction:
谢谢你，我都给忙忘记了。
Response:
哈哈，我还知道今天是周杰伦的生日呢。
Instruction:
真的吗？我最喜欢听他的歌了，他是我的偶像呢。
Response:
真的呢，他的歌声确实是很独特，还获得过四届新加坡金曲奖最受欢迎男歌手奖呢。
Instruction:
高中时候很喜欢他的歌，后面只是很少关注他了。
###output: 那他演唱的『免费教学录像带』很赞呢，明显还是这么天马行空，里面各种可爱的小东东，推荐给你。


In [137]:
print('###input',processed_datasets['train']['input'][4])
print('###output:',processed_datasets['train']['target'][4])

###input Instruction:
你好，今天是几号了？
Response:
你好，今天是2018年1月18日。
Instruction:
谢谢你，我都给忙忘记了。
Response:
哈哈，我还知道今天是周杰伦的生日呢。
Instruction:
真的吗？我最喜欢听他的歌了，他是我的偶像呢。
Response:
真的呢，他的歌声确实是很独特，还获得过四届新加坡金曲奖最受欢迎男歌手奖呢。
Instruction:
高中时候很喜欢他的歌，后面只是很少关注他了。
Response:
那他演唱的『免费教学录像带』很赞呢，明显还是这么天马行空，里面各种可爱的小东东，推荐给你。
Instruction:
是新出来的歌么？看这个歌名不错啊。
###output: 很多人都喜欢听呢。


In [138]:
print('###input',processed_datasets['train']['input'][5])
print('###output:',processed_datasets['train']['target'][5])

###input Instruction:
你好，今天是几号了？
Response:
你好，今天是2018年1月18日。
Instruction:
谢谢你，我都给忙忘记了。
Response:
哈哈，我还知道今天是周杰伦的生日呢。
Instruction:
真的吗？我最喜欢听他的歌了，他是我的偶像呢。
Response:
真的呢，他的歌声确实是很独特，还获得过四届新加坡金曲奖最受欢迎男歌手奖呢。
Instruction:
高中时候很喜欢他的歌，后面只是很少关注他了。
Response:
那他演唱的『免费教学录像带』很赞呢，明显还是这么天马行空，里面各种可爱的小东东，推荐给你。
Instruction:
是新出来的歌么？看这个歌名不错啊。
Response:
很多人都喜欢听呢。
Instruction:
说的我都想听一听了。
###output: 那你现在有时间吗？我可以为你播放。


In [139]:
print('###input',processed_datasets['train']['input'][6])
print('###output:',processed_datasets['train']['target'][6])

###input Instruction:
你好，今天是几号了？
Response:
你好，今天是2018年1月18日。
Instruction:
谢谢你，我都给忙忘记了。
Response:
哈哈，我还知道今天是周杰伦的生日呢。
Instruction:
真的吗？我最喜欢听他的歌了，他是我的偶像呢。
Response:
真的呢，他的歌声确实是很独特，还获得过四届新加坡金曲奖最受欢迎男歌手奖呢。
Instruction:
高中时候很喜欢他的歌，后面只是很少关注他了。
Response:
那他演唱的『免费教学录像带』很赞呢，明显还是这么天马行空，里面各种可爱的小东东，推荐给你。
Instruction:
是新出来的歌么？看这个歌名不错啊。
Response:
很多人都喜欢听呢。
Instruction:
说的我都想听一听了。
Response:
那你现在有时间吗？我可以为你播放。
Instruction:
嗯嗯，快点播放吧，我都有点迫不及待了。
###output: 好的呢，已经为你播放，请你欣赏。


In [140]:
print('###input',processed_datasets['train']['input'][7])
print('###output:',processed_datasets['train']['target'][7])

###input Instruction:
你好，今天是几号了？
Response:
你好，今天是2018年1月18日。
Instruction:
谢谢你，我都给忙忘记了。
Response:
哈哈，我还知道今天是周杰伦的生日呢。
Instruction:
真的吗？我最喜欢听他的歌了，他是我的偶像呢。
Response:
真的呢，他的歌声确实是很独特，还获得过四届新加坡金曲奖最受欢迎男歌手奖呢。
Instruction:
高中时候很喜欢他的歌，后面只是很少关注他了。
Response:
那他演唱的『免费教学录像带』很赞呢，明显还是这么天马行空，里面各种可爱的小东东，推荐给你。
Instruction:
是新出来的歌么？看这个歌名不错啊。
Response:
很多人都喜欢听呢。
Instruction:
说的我都想听一听了。
Response:
那你现在有时间吗？我可以为你播放。
Instruction:
嗯嗯，快点播放吧，我都有点迫不及待了。
Response:
好的呢，已经为你播放，请你欣赏。
Instruction:
真不愧是周董的歌啊，还是熟悉的感觉。不说了，我上班去了，回来聊。
###output: 好的，拜拜~


In [141]:
print('###input',processed_datasets['train']['input'][8])
print('###output:',processed_datasets['train']['target'][8])

###input Instruction:
谢娜是哪里的人啊？
###output: 她是四川的哦！


In [142]:
processed_datasets

{'train': Dataset({
     features: ['input', 'target'],
     num_rows: 30225
 }),
 'validation': Dataset({
     features: ['input', 'target'],
     num_rows: 4297
 }),
 'test': Dataset({
     features: ['input', 'target'],
     num_rows: 5723
 })}

In [8]:
def tokenize_function(examples):
    # Adjust "max_length" and "padding" according to your needs
    model_inputs = tokenizer(examples['input'], padding="max_length", truncation=True, max_length=512)
    # Tokenize targets, ensure to set padding and truncation to match your model's requirements
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['target'], padding="max_length", truncation=True, max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = {split: processed_datasets[split].map(tokenize_function, batched=True) for split in processed_datasets.keys()}

  0%|          | 0/31 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [8]:
tokenized_datasets

{'train': Dataset({
     features: ['input', 'target', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 30225
 }),
 'validation': Dataset({
     features: ['input', 'target', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 4297
 }),
 'test': Dataset({
     features: ['input', 'target', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
     num_rows: 5723
 })}

**Load Model**

In [9]:
import torch
from transformers import BartForConditionalGeneration

device="cuda:0"

model = BartForConditionalGeneration.from_pretrained(
    model_name,
    device_map=device,
    #quantization_config=quantization_config,
    low_cpu_mem_usage=True,
    #trust_remote_code=True
)
model.half()
model.config.use_cache = False

model.safetensors:   0%|          | 0.00/561M [00:00<?, ?B/s]

**Training Parameters**

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True, return_tensors="pt")

2024-04-08 10:00:09.776756: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 10:00:09.776870: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 10:00:09.948376: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
from peft import LoraConfig

peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    #task_type="CAUSAL_LM",
    task_type="SEQ_2_SEQ_LM",
)

In [12]:
from peft import get_peft_model

model = get_peft_model(model, peft_params)
model.print_trainable_parameters()

trainable params: 3,538,944 || all params: 143,731,968 || trainable%: 2.462182943184915


In [13]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    num_train_epochs=3,
    # 5 -> Not Good
    # 3 -> Quite Good
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    optim='paged_adamw_8bit',
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-3, #2e-3 > 2e-4 > 2e-2
    #1e-3 0.29 0.52
    #2e-3 0.3 0.47
    #3e-3 0.33 0.28 Winner But Response are strange
    #4e-3 0.36 1.18
    weight_decay=0.003,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type='constant',
    report_to='tensorboard',
    predict_with_generate=True
)

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [15]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.model.model.decoder.embed_tokens.weight', 'base_model.model.model.encoder.embed_tokens.weight', 'base_model.model.model.shared.weight', 'base_model.model.lm_head.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

In [16]:
model = model.to(device)
model.eval()

inputs = tokenizer("范冰冰多重啊？", return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to(device), max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


范 冰 冰 的 身 高 是 170cm 。


**Evaluation**

In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['conversation'],
        num_rows: 6618
    })
    validation: Dataset({
        features: ['conversation'],
        num_rows: 946
    })
    test: Dataset({
        features: ['conversation'],
        num_rows: 2626
    })
})

**Save Model**

In [21]:
trainer.model.save_pretrained("new_model")

In [5]:
model.save_pretrained("new_model2")

NameError: name 'model' is not defined

In [19]:
tokenizer.save_pretrained("new_model")

('new_model2/tokenizer_config.json',
 'new_model2/special_tokens_map.json',
 'new_model2/vocab.txt',
 'new_model2/added_tokens.json')